In [3]:
import requests
# from pyden.connect.gsearch_api.google import google
# from pyden.newspaper.newspaper import Article
from newspaper import Article
# from pyden.google_api.search import gsearch

import time
import unidecode


In [4]:
toscrape = ["https://amwalalghad.com", "https://www.mubasher.info/markets/ADX/stocks/NBAD/news", "https://amwalalghad.com", "https://www.alarabiya.net/ar/tools"]


In [5]:
# define using se scraper and parsing with newspaper 3k
class Object(object):
    pass
def gsearch(term, pages = 1):
    term = term.replace("\"","").replace("\'","").replace("-", "")
    term += " -filetype:pdf"  #TODO: be able to read pdfs
    data = '{ "browser_config": { "random_user_agent": true },"scrape_config": { "search_engine": "google","keywords": ["'+term+'"],"num_pages": '+str(pages)+'}}'
    header = {'Content-Type': 'application/json'}
    results = requests.post("http://0.0.0.0:3000", data = data, headers = header).json()
    output = []

    for num in range(int(pages)):
        if str(num+1) in results['results'][term]:
            output.extend([i for i in results['results'][term][str(num+1)]['results']])
    return output

def search(query, link = ""):
    unaccented_string = unidecode.unidecode(query)
    query_outs = gsearch(unaccented_string)
    titles = []
    labels = []
    para = []
    rm = []
    outputs = [Object() for _ in query_outs]

    for i, qo in enumerate(query_outs):
        query = outputs[i]; query.link = qo['link']
        # if not query.link or query.link.replace('/', u'\u2215') == link[0]:  # the links have '/' replaced
        #     rm.append(i)
        #     continue
        art = Article(query.link)  # if you are an article, parse and
        try:
            start = time.time()
            art.download()
            if not art.html:
                rm.append(i)
                continue
            print(query.link)
            print("took " + str(time.time() - start))
            art.parse()  # some articles cannot be parsed
            para.append(art.text.split("\n\n"))  # this makes paras a list of lists
            toadd = query.link.replace('/', u'\u2215')
            if len(query.link) > 255:
                toadd = toadd[:255]
            labels.append(toadd)  # titles are the labels that things are saved by
            query.title = art.title
            if art.title == "":
                query.title = qo[i]['title']  # this is just going to be what the title was on google
            query.sources = art.source_url
            query.authors = art.authors
            if art.publish_date:
                query.date = art.publish_date.strftime('%m/%d/%Y')
        except Exception as e:
            print(e)  # if u cant load it uhhhh just dont do anything
    for index in sorted(rm, reverse=True):
        del outputs[index]
    return outputs, labels, para, titles



In [9]:
results = search("site:" + toscrape[0] + " fab")

text/html; charset=UTF-8


text/html; charset=UTF-8
text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


In [10]:
print(results[2]) 

In [75]:
model_path = "/home/raymond/finBERT/models/trc2/"

In [68]:
# path to trc2 pytorch model # for some reason flask app doesn't work
paragraph = results[2][1][1]
# sample = '{"text": ' + paragraph + '}'
sample = {"text": paragraph}
header = {'Content-Type': 'application/json'}
def get_sentiment(text):
    results = requests.post("http://0.0.0.0:8080", json = sample, headers = header)
print(type(get_sentiment(sample)))

In [76]:
# paste in flask app prediction code here, but remove the flask
import sys
sys.path.insert(0, "/home/raymond/finBERT")
from finbert.finbert import predict
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(model_path,num_labels=3,cache_dir=None)


09/07/2020 21:54:38 - INFO - pytorch_pretrained_bert.modeling -   loading archive file /home/raymond/finBERT/models/trc2/
09/07/2020 21:54:38 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

09/07/2020 21:54:40 - INFO - pytorch_pretrained_bert.modeling -   Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
09/07/2020 21:54:40 - INFO - pytorch_pretrained_bert.modeling -   Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.p

In [82]:
# take a list of sentiment analysis of all the sentences that have the stock name
sentiments = []
for article in results[2]:
    per_article = []
    for paragraph in article:
        if paragraph:
            per_article.append(predict(paragraph,model))
    sentiments.append(per_article)

09/07/2020 21:58:43 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/raymond/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
09/07/2020 21:58:43 - INFO - finbert.utils -   *** Example ***
09/07/2020 21:58:43 - INFO - finbert.utils -   guid: 0
09/07/2020 21:58:43 - INFO - finbert.utils -   tokens: [CLS] egypt has tapped emirates n ##b ##d and first abu dhabi bank ( fa ##b ) to arrange a loan worth more than $ 1 billion as it tries to plug its financing gap caused by the co ##vid - 19 pan ##de ##mic , reuters reports , citing two sources familiar with the matter . [SEP]
09/07/2020 21:58:43 - INFO - finbert.utils -   input_ids: 101 5279 2038 10410 14041 1050 2497 2094 1998 2034 8273 23153 2924 1006 6904 2497 1007 2000 13621 1037 5414 4276 2062 2084 1002 1015 45

09/07/2020 21:58:45 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:45 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:45 - INFO - finbert.utils -   label: None (id = 9090)
09/07/2020 21:58:46 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/raymond/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
09/07/2020 21:58:46 - INFO - finbert.utils -   *** Example ***
09/07/2020 21:58:46 - INFO - finbert.utils -   guid: 0
09/07/2020 21:58:46 - INFO - finbert.utils -   tokens: [CLS] in saudi arabia th

09/07/2020 21:58:49 - INFO - finbert.utils -   tokens: [CLS] in abu dhabi the index rose 1 . 1 percent , snapping a three - day losing streak with market heavyweight first abu dhabi bank ( fa ##b ) advancing 2 percent and abu dhabi commercial bank gaining 0 . 3 percent . [SEP]
09/07/2020 21:58:49 - INFO - finbert.utils -   input_ids: 101 1999 8273 23153 1996 5950 3123 1015 1012 1015 3867 1010 15790 1037 2093 1011 2154 3974 9039 2007 3006 8366 2034 8273 23153 2924 1006 6904 2497 1007 10787 1016 3867 1998 8273 23153 3293 2924 8550 1014 1012 1017 3867 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:49 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:49 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:49 - I

09/07/2020 21:58:52 - INFO - finbert.utils -   *** Example ***
09/07/2020 21:58:52 - INFO - finbert.utils -   guid: 0
09/07/2020 21:58:52 - INFO - finbert.utils -   tokens: [CLS] source : reuters [SEP]
09/07/2020 21:58:52 - INFO - finbert.utils -   input_ids: 101 3120 1024 26665 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:52 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:52 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:52 - INFO - finbert.utils -   label: None (id = 9090)
09/07/2020 21:58:52 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-b

09/07/2020 21:58:55 - INFO - finbert.utils -   *** Example ***
09/07/2020 21:58:55 - INFO - finbert.utils -   guid: 0
09/07/2020 21:58:55 - INFO - finbert.utils -   tokens: [CLS] the united arab emirates ’ largest sha ##ria - compliant bank is in talks with shareholders of un ##list ##ed dubai - based no ##or bank over a possible acquisition of the lend ##er , three sources familiar with the matter told reuters last week . [SEP]
09/07/2020 21:58:55 - INFO - finbert.utils -   input_ids: 101 1996 2142 5424 14041 1521 2922 21146 4360 1011 24577 2924 2003 1999 7566 2007 15337 1997 4895 9863 2098 11558 1011 2241 2053 2953 2924 2058 1037 2825 7654 1997 1996 18496 2121 1010 2093 4216 5220 2007 1996 3043 2409 26665 2197 2733 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:55 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:58:55 - INFO - finbert.utils - 

...

09/07/2020 21:59:04 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/raymond/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
09/07/2020 21:59:05 - INFO - finbert.utils -   *** Example ***
09/07/2020 21:59:05 - INFO - finbert.utils -   guid: 0
09/07/2020 21:59:05 - INFO - finbert.utils -   tokens: [CLS] the index was trading 0 . 2 percent lower with national commercial bank down 0 . 9 percent and ri ##yad bank down almost 1 percent . [SEP]
09/07/2020 21:59:05 - INFO - finbert.utils -   input_ids: 101 1996 5950 2001 6202 1014 1012 1016 3867 2896 2007 2120 3293 2924 2091 1014 1012 1023 3867 1998 15544 25152 2924 2091 2471 1015 3867 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:59:05 - INFO - finbert.utils -   input

09/07/2020 21:59:08 - INFO - finbert.utils -   *** Example ***
09/07/2020 21:59:08 - INFO - finbert.utils -   guid: 0
09/07/2020 21:59:08 - INFO - finbert.utils -   tokens: [CLS] the abu dhabi index lost 1 percent , with first abu dhabi bank shed ##ding 1 . 5 percent before announcing its first - quarter results later in the day . [SEP]
09/07/2020 21:59:08 - INFO - finbert.utils -   input_ids: 101 1996 8273 23153 5950 2439 1015 3867 1010 2007 2034 8273 23153 2924 8328 4667 1015 1012 1019 3867 2077 13856 2049 2034 1011 4284 3463 2101 1999 1996 2154 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:59:08 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:59:08 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


09/07/2020 21:59:12 - INFO - finbert.utils -   *** Example ***
09/07/2020 21:59:12 - INFO - finbert.utils -   guid: 0
09/07/2020 21:59:12 - INFO - finbert.utils -   tokens: [CLS] qatar ’ s blue - chip index was down 0 . 1 percent . [SEP]
09/07/2020 21:59:12 - INFO - finbert.utils -   input_ids: 101 12577 1521 1055 2630 1011 9090 5950 2001 2091 1014 1012 1015 3867 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:59:12 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:59:12 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:59:12 - INFO - finbert.utils -   label: None (id = 9090)
09/07/2020 21:59:12 - INFO - pytorch_pretrained_bert.tokenization -   loading vocab

09/07/2020 21:59:15 - INFO - finbert.utils -   input_ids: 101 1999 2089 1010 6904 2497 2404 7566 2006 2907 2349 2000 3697 3006 3785 5533 2011 1996 21887 23350 6090 3207 7712 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:59:15 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:59:15 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/07/2020 21:59:15 - INFO - finbert.utils -   label: None (id = 9090)
09/07/2020 21:59:15 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/raymond/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.5

09/07/2020 21:59:18 - INFO - finbert.utils -   *** Example ***
09/07/2020 21:59:18 - INFO - finbert.utils -   guid: 0
09/07/2020 21:59:18 - INFO - finbert.utils -   tokens: [CLS] “ despite the inc ##omp ##let ##ion of the sale deal due to the exceptional by the corona ##virus crisis which has led to the suspension of all talks , we would like to thank first abu dhabi bank for its efforts and creative discussions . ” said bank audi ’ s chief financial officer tame ##r g ##ha ##zal ##eh . [SEP]
09/07/2020 21:59:18 - INFO - finbert.utils -   input_ids: 101 1523 2750 1996 4297 25377 7485 3258 1997 1996 5096 3066 2349 2000 1996 11813 2011 1996 21887 23350 5325 2029 2038 2419 2000 1996 8636 1997 2035 7566 1010 2057 2052 2066 2000 4067 2034 8273 23153 2924 2005 2049 4073 1998 5541 10287 1012 1524 2056 2924 20075 1521 1055 2708 3361 2961 24763 2099 1043 3270 16739 11106 1012 102
09/07/2020 21:59:18 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [87]:
sentiments[0][0]

,sentence,logit,prediction,sentiment_score
0,Egypt has tapped Emirates NBD and First Abu Dh...,"[0.29286325, 0.2198136, 0.48732308]",neutral,0.07305


In [86]:
results[0][0].date

'06/17/2020'